In [1]:
import os
from dotenv import load_dotenv
from openai import OpenAI

# Environment setup
load_dotenv(override=True)
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')

# Initialize OpenAI
openai = OpenAI()
OPENAI_MODEL = "gpt-4o-mini"

# System message for C++ conversion
system_message = (
    "You are an assistant that reimplements Python code in high performance C++ for an M1 Mac. "
    "Respond only with C++ code; use comments sparingly. "
    "Produce identical output in the fastest possible time. "
    "Keep random number generators identical to match Python results."
)

def user_prompt_for(python):
    return (
        "Rewrite this Python code in C++ with the fastest implementation that produces identical output. "
        "Respond only with C++ code; use minimal comments. "
        "Ensure correct number types to avoid overflows. Include necessary C++ headers like iomanip.\n\n"
        f"{python}"
    )

def messages_for(python):
    return [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_prompt_for(python)}
    ]

def write_output(cpp):
    code = cpp.replace("```cpp", "").replace("```", "")
    with open("optimized.cpp", "w") as f:
        f.write(code)

def optimize_gpt(python):
    stream = openai.chat.completions.create(
        model=OPENAI_MODEL,
        messages=messages_for(python),
        stream=True
    )
    reply = ""
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        print(fragment, end='', flush=True)
    write_output(reply)
    return reply

# Sample Python programs
pi = """
import time

def calculate(iterations, param1, param2):
    result = 1.0
    for i in range(1, iterations+1):
        j = i * param1 - param2
        result -= (1/j)
        j = i * param1 + param2
        result += (1/j)
    return result

start_time = time.time()
result = calculate(100_000_000, 4, 1) * 4
end_time = time.time()

print(f"Result: {result:.12f}")
print(f"Execution Time: {(end_time - start_time):.6f} seconds")
"""

python_hard = """
def lcg(seed, a=1664525, c=1013904223, m=2**32):
    value = seed
    while True:
        value = (a * value + c) % m
        yield value
        
def max_subarray_sum(n, seed, min_val, max_val):
    lcg_gen = lcg(seed)
    random_numbers = [next(lcg_gen) % (max_val - min_val + 1) + min_val for _ in range(n)]
    max_sum = float('-inf')
    for i in range(n):
        current_sum = 0
        for j in range(i, n):
            current_sum += random_numbers[j]
            if current_sum > max_sum:
                max_sum = current_sum
    return max_sum

def total_max_subarray_sum(n, initial_seed, min_val, max_val):
    total_sum = 0
    lcg_gen = lcg(initial_seed)
    for _ in range(20):
        seed = next(lcg_gen)
        total_sum += max_subarray_sum(n, seed, min_val, max_val)
    return total_sum

n = 10000
initial_seed = 42
min_val = -10
max_val = 10

import time
start_time = time.time()
result = total_max_subarray_sum(n, initial_seed, min_val, max_val)
end_time = time.time()

print("Total Maximum Subarray Sum (20 runs):", result)
print("Execution Time: {:.6f} seconds".format(end_time - start_time))
"""

# Execute and optimize
print("Optimizing pi calculation...")
exec(pi)
optimize_gpt(pi)

print("\nOptimizing max subarray sum...")
exec(python_hard)
optimize_gpt(python_hard)

Optimizing pi calculation...
Result: 3.141592658589
Execution Time: 7.657818 seconds
```cpp
#include <iostream>
#include <iomanip>
#include <chrono>

double calculate(int64_t iterations, double param1, double param2) {
    double result = 1.0;
    for (int64_t i = 1; i <= iterations; ++i) {
        double j = i * param1 - param2;
        result -= (1.0 / j);
        j = i * param1 + param2;
        result += (1.0 / j);
    }
    return result;
}

int main() {
    auto start_time = std::chrono::high_resolution_clock::now();
    
    double result = calculate(100000000, 4.0, 1.0) * 4.0;
    
    auto end_time = std::chrono::high_resolution_clock::now();
    std::chrono::duration<double> execution_time = end_time - start_time;
    
    std::cout << std::fixed << std::setprecision(12);
    std::cout << "Result: " << result << std::endl;
    std::cout << "Execution Time: " << execution_time.count() << " seconds" << std::endl;

    return 0;
}
```
Optimizing max subarray sum...
Total Maximum

'```cpp\n#include <iostream>\n#include <iomanip>\n#include <vector>\n#include <limits>\n#include <chrono>\n\nclass LCG {\npublic:\n    LCG(uint32_t seed, uint32_t a = 1664525, uint32_t c = 1013904223, uint32_t m = 4294967296) \n        : value(seed), a(a), c(c), m(m) {}\n\n    uint32_t next() {\n        value = (a * value + c) % m;\n        return value;\n    }\n\nprivate:\n    uint32_t value, a, c, m;\n};\n\nint64_t max_subarray_sum(size_t n, uint32_t seed, int min_val, int max_val) {\n    LCG lcg(seed);\n    std::vector<int64_t> random_numbers(n);\n    \n    for (size_t i = 0; i < n; ++i) {\n        random_numbers[i] = lcg.next() % (max_val - min_val + 1) + min_val;\n    }\n    \n    int64_t max_sum = std::numeric_limits<int64_t>::min();\n    \n    for (size_t i = 0; i < n; ++i) {\n        int64_t current_sum = 0;\n        for (size_t j = i; j < n; ++j) {\n            current_sum += random_numbers[j];\n            if (current_sum > max_sum) {\n                max_sum = current_sum;\n